<a href="https://colab.research.google.com/github/mahemon/PySpark-practice/blob/main/Partition_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


https://www.sqlshack.com/sql-lag-function-overview-and-examples/  


https://sparkbyexamples.com/pyspark/pyspark-window-functions/

https://stackoverflow.com/questions/57525149/spark-how-to-aggregate-reduce-records-based-on-time-difference


In [71]:
#spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
simpleData = spark.createDataFrame(data = simpleData, schema = columns)
simpleData.printSchema()
simpleData.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [73]:
from pyspark.sql.functions import rank,dense_rank

In [74]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("department").orderBy("salary")

simpleData.withColumn("row_number",dense_rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |1         |
|Robert       |Sales     |4100  |2         |
|Saif         |Sales     |4100  |2         |
|Michael      |Sales     |4600  |3         |
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
+-------------+----------+------+----------+



In [75]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("department").orderBy("salary")
simpleData.withColumn("row_number",rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |1         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |3         |
|Michael      |Sales     |4600  |5         |
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
+-------------+----------+------+----------+



In [76]:
from pyspark.sql.functions import lag    
windowSpec  = Window.partitionBy("department").orderBy("salary")
simpleData.withColumn("lag",lag("salary",2).over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary| lag|
+-------------+----------+------+----+
|        James|     Sales|  3000|null|
|        James|     Sales|  3000|null|
|       Robert|     Sales|  4100|3000|
|         Saif|     Sales|  4100|3000|
|      Michael|     Sales|  4600|4100|
|        Maria|   Finance|  3000|null|
|        Scott|   Finance|  3300|null|
|          Jen|   Finance|  3900|3000|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|null|
+-------------+----------+------+----+



In [94]:
#pyspark.sql.functions.lag(col, offset=1, default=None)    #offsetint -> number of row to extend  default-> default value
from pyspark.sql.functions import lag,row_number 
from pyspark.sql.window import Window   
windowSpec  = Window.partitionBy("department").orderBy("salary")
simpleData.withColumn("lag",lag("salary",1,1).over(windowSpec)).show()

+-------------+----------+------+------+----+
|employee_name|department|salary|newcol| lag|
+-------------+----------+------+------+----+
|        James|     Sales|  3000|  null|   1|
|        James|     Sales|  3000|  3000|3000|
|       Robert|     Sales|  4100|  3000|3000|
|         Saif|     Sales|  4100|  4100|4100|
|      Michael|     Sales|  4600|  4100|4100|
|        Maria|   Finance|  3000|  null|   1|
|        Scott|   Finance|  3300|  3000|3000|
|          Jen|   Finance|  3900|  3300|3300|
|        Kumar| Marketing|  2000|  null|   1|
|         Jeff| Marketing|  3000|  2000|2000|
+-------------+----------+------+------+----+



In [78]:
"""lead"""
from pyspark.sql.functions import lead    
simpleData.withColumn("lead",lead("salary",2).over(windowSpec)).show()

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|        James|     Sales|  3000|4100|
|        James|     Sales|  3000|4100|
|       Robert|     Sales|  4100|4600|
|         Saif|     Sales|  4100|null|
|      Michael|     Sales|  4600|null|
|        Maria|   Finance|  3000|3900|
|        Scott|   Finance|  3300|null|
|          Jen|   Finance|  3900|null|
|        Kumar| Marketing|  2000|null|
|         Jeff| Marketing|  3000|null|
+-------------+----------+------+----+



In [82]:
from pyspark.sql.functions import lag    
windowSpec  = Window.partitionBy("department").orderBy("salary")
simpleData=simpleData.withColumn("newcol",lag("salary",1).over(windowSpec))

In [83]:
simpleData.show(2)

+-------------+----------+------+------+
|employee_name|department|salary|newcol|
+-------------+----------+------+------+
|        James|     Sales|  3000|  null|
|        James|     Sales|  3000|  3000|
+-------------+----------+------+------+
only showing top 2 rows



In [84]:
simpleData.withColumn("diff",simpleData['newcol']-simpleData['salary']).show()

+-------------+----------+------+------+-----+
|employee_name|department|salary|newcol| diff|
+-------------+----------+------+------+-----+
|        James|     Sales|  3000|  null| null|
|        James|     Sales|  3000|  3000|    0|
|       Robert|     Sales|  4100|  3000|-1100|
|         Saif|     Sales|  4100|  4100|    0|
|      Michael|     Sales|  4600|  4100| -500|
|        Maria|   Finance|  3000|  null| null|
|        Scott|   Finance|  3300|  3000| -300|
|          Jen|   Finance|  3900|  3300| -600|
|        Kumar| Marketing|  2000|  null| null|
|         Jeff| Marketing|  3000|  2000|-1000|
+-------------+----------+------+------+-----+



Speed DataSet

In [92]:
sDb = ((1,1538204192 , 44.55), \
       (1, 1538204193, 47.20), \
     (1,1538204194 , 42.14), \
     (1,1538204195 , 39.20), \
     (1, 1538204196, 35.30), \
     (1, 1538204197, 32.22), \
     (1, 1538204198, 34.80), \
    (1, 1538204199, 37.10), \
    (1, 1538204221, 55.30), \
    (1, 1538204222, 57.20), \
  (1, 1538204223, 54.60), \
  (1, 1538204224, 52.15), \
  (1, 1538204224, 49.27), \
   (1, 1538204225, 47.89), \
   (1, 1538204226, 50.57), \
  (1, 1538204227, 53.72))
 
columns= ["trip_id", "timestamp", "speed"]
sDb = spark.createDataFrame(data = sDb, schema = columns)
sDb.printSchema()
sDb.show(truncate=False)

root
 |-- trip_id: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- speed: double (nullable = true)

+-------+----------+-----+
|trip_id|timestamp |speed|
+-------+----------+-----+
|1      |1538204192|44.55|
|1      |1538204193|47.2 |
|1      |1538204194|42.14|
|1      |1538204195|39.2 |
|1      |1538204196|35.3 |
|1      |1538204197|32.22|
|1      |1538204198|34.8 |
|1      |1538204199|37.1 |
|1      |1538204221|55.3 |
|1      |1538204222|57.2 |
|1      |1538204223|54.6 |
|1      |1538204224|52.15|
|1      |1538204224|49.27|
|1      |1538204225|47.89|
|1      |1538204226|50.57|
|1      |1538204227|53.72|
+-------+----------+-----+



In [203]:
from pyspark.sql.functions import lag,row_number,when,lead,concat
from pyspark.sql.window import Window 

In [154]:
overColumns = Window.partitionBy("trip_id").orderBy("timestamp")
breaksDF = sDb.withColumn("speeddiff", sDb['speed']-lag(sDb['speed'], 1,0).over(overColumns))

In [155]:
breaksDF.show(5)

+-------+----------+-----+-------------------+
|trip_id| timestamp|speed|          speeddiff|
+-------+----------+-----+-------------------+
|      1|1538204192|44.55|              44.55|
|      1|1538204193| 47.2| 2.6500000000000057|
|      1|1538204194|42.14| -5.060000000000002|
|      1|1538204195| 39.2|-2.9399999999999977|
|      1|1538204196| 35.3|-3.9000000000000057|
+-------+----------+-----+-------------------+
only showing top 5 rows



In [159]:
import pyspark.sql.functions as F
brEvent = breaksDF.withColumn('breaking',F.when((breaksDF['speeddiff'] < 1), 1).otherwise(0))

In [160]:
brEvent.show()

+-------+----------+-----+-------------------+--------+
|trip_id| timestamp|speed|          speeddiff|breaking|
+-------+----------+-----+-------------------+--------+
|      1|1538204192|44.55|              44.55|       0|
|      1|1538204193| 47.2| 2.6500000000000057|       0|
|      1|1538204194|42.14| -5.060000000000002|       1|
|      1|1538204195| 39.2|-2.9399999999999977|       1|
|      1|1538204196| 35.3|-3.9000000000000057|       1|
|      1|1538204197|32.22|-3.0799999999999983|       1|
|      1|1538204198| 34.8| 2.5799999999999983|       0|
|      1|1538204199| 37.1| 2.3000000000000043|       0|
|      1|1538204221| 55.3| 18.199999999999996|       0|
|      1|1538204222| 57.2| 1.9000000000000057|       0|
|      1|1538204223| 54.6|-2.6000000000000014|       1|
|      1|1538204224|52.15| -2.450000000000003|       1|
|      1|1538204224|49.27|-2.8799999999999955|       1|
|      1|1538204225|47.89|-1.3800000000000026|       1|
|      1|1538204226|50.57| 2.6799999999999997|  

In [182]:
outputDF = brEvent.withColumn("breakevent", F.when((brEvent['breaking'] - lag(brEvent['breaking'], 1).over(overColumns)) == 1, "start of break").when((outputDF['breaking'] - lead(outputDF['breaking'], 1).over(overColumns)) == 1, "end of break"))

In [184]:
#outputDF2 = outputDF.withColumn("breakevent", F.when((outputDF['breaking'] - lead(outputDF['breaking'], 1).over(overColumns)) == 1, "end of break"))

In [185]:
outputDF.show()

+-------+----------+-----+-------------------+--------+--------------+
|trip_id| timestamp|speed|          speeddiff|breaking|    breakevent|
+-------+----------+-----+-------------------+--------+--------------+
|      1|1538204192|44.55|              44.55|       0|          null|
|      1|1538204193| 47.2| 2.6500000000000057|       0|          null|
|      1|1538204194|42.14| -5.060000000000002|       1|start of break|
|      1|1538204195| 39.2|-2.9399999999999977|       1|          null|
|      1|1538204196| 35.3|-3.9000000000000057|       1|          null|
|      1|1538204197|32.22|-3.0799999999999983|       1|  end of break|
|      1|1538204198| 34.8| 2.5799999999999983|       0|          null|
|      1|1538204199| 37.1| 2.3000000000000043|       0|          null|
|      1|1538204221| 55.3| 18.199999999999996|       0|          null|
|      1|1538204222| 57.2| 1.9000000000000057|       0|          null|
|      1|1538204223| 54.6|-2.6000000000000014|       1|start of break|
|     

In [192]:
outputDF.filter("breakevent is not null").select("trip_id", "timestamp", "speed", "breakevent").show()

+-------+----------+-----+--------------+
|trip_id| timestamp|speed|    breakevent|
+-------+----------+-----+--------------+
|      1|1538204194|42.14|start of break|
|      1|1538204197|32.22|  end of break|
|      1|1538204223| 54.6|start of break|
|      1|1538204225|47.89|  end of break|
+-------+----------+-----+--------------+



In [205]:
from pyspark.sql.functions import first,last

In [218]:
outputDF.filter("breakevent is not null").withColumn("breakID", when(outputDF['breakevent'] == "start of break", concat(outputDF['trip_id'],outputDF['timestamp'])).when(outputDF['breakevent'] == "end of break", concat(outputDF['trip_id'], lag(outputDF['timestamp'], 1).over(overColumns)))).show()

+-------+----------+-----+-------------------+--------+--------------+-----------+
|trip_id| timestamp|speed|          speeddiff|breaking|    breakevent|    breakID|
+-------+----------+-----+-------------------+--------+--------------+-----------+
|      1|1538204194|42.14| -5.060000000000002|       1|start of break|11538204194|
|      1|1538204197|32.22|-3.0799999999999983|       1|  end of break|11538204194|
|      1|1538204223| 54.6|-2.6000000000000014|       1|start of break|11538204223|
|      1|1538204225|47.89|-1.3800000000000026|       1|  end of break|11538204223|
+-------+----------+-----+-------------------+--------+--------------+-----------+



In [221]:
outputDF.filter("breakevent is not null").withColumn("breakID", when(outputDF['breakevent'] == "start of break", concat(outputDF['trip_id'],outputDF['timestamp'])).when(outputDF['breakevent'] == "end of break", concat(outputDF['trip_id'], lag(outputDF['timestamp'], 1).over(overColumns)))).groupBy("breakID").agg(F.first(outputDF['timestamp']), F.last(outputDF['timestamp']), F.first(outputDF['speed']), F.last(outputDF['speed'])).show()

+-----------+-----------------------+----------------------+-------------------+------------------+
|    breakID|first(timestamp, false)|last(timestamp, false)|first(speed, false)|last(speed, false)|
+-----------+-----------------------+----------------------+-------------------+------------------+
|11538204223|             1538204223|            1538204225|               54.6|             47.89|
|11538204194|             1538204194|            1538204197|              42.14|             32.22|
+-----------+-----------------------+----------------------+-------------------+------------------+

